### Goal:
#### Produce a spreadsheet containing radiance value averages per band. Each image is already cropped 
##### Objectives:
* Loop through network drive and identify a list of STD images
* Generate spreadsheets consisting of radiance averages for each image's differnt exposure 
* Notes: The item folders in NAS 2 do not contain any meta data, this data is found from NAS 1

#### Root directories 
##### Note: 5ms, 10ms, 25ms bin and hdr sets in each STD folder
##### IOP1_Sep2020
* "Z:\Data\St_Supery\IOP1_Sep2020\Tripod"

##### IOP2_Oct2020
* "Z:\Data\St_Supery\IOP2_Oct2020\Tripod"
##### IOP3_July2021
* "Z:\Data\St_Supery\IOP3_July2021\Tripod"
##### IOP4_August2021
* "Z:\Data\St_Supery\IOP4_August2021\Tripod"
##### IOP5_Sep2021
* "Z:\Data\St_Supery\IOP5_Sep2021\Tripod"
##### IOP6_October2021
* "Z:\Data\St_Supery\IOP6_October2021\Tripod"

In [24]:
# Dependencies which need to be installed the first time before running:

# These can be uncommented and will be executed in this cell:
# !{sys.executable} -m pip install spectral
# !{sys.executable} -m pip install wxpython

# These same commands can be installed into your python interpreter libraries by running:
# in terminal: pip install spectral
# in terminal: pip install wxpython
 




from PIL import Image
from spectral import *
import spectral.io.envi as envi
import numpy as np
import pandas as pd
from os import listdir
from os.path import isfile, isdir, join
# sys.path.append("../Utilities")
from HDRprocess import *
import glob
import winsound

# function to grab a pair of bin and hdr:
def Extract_Files_By_Exposure(path,exposure):
    """ Helps identify BIN and HDR files, as genereated by Per Class Mira software, returns (0) """
    try:
        files = listdir(path)
        # Filter the BIN and HDR values that have selected exposure
        bin = [x for x in files if x.find(exposure) != -1 and x[-3:] == 'bin'][0]
        hdr = [x for x in files if x.find(exposure) != -1 and x[-3:] == 'hdr'][0]
        bin = join(path,bin)
        hdr = join(path,hdr)
        return (bin,hdr)
    except :
        return (0,0)

# The wavelength values consistent among all exposure lengths
wv = [500,501.7,503.3,505,506.7,508.4,510,511.7,513.4,515.1,516.7,518.4,520.1,521.8,523.4,525.1,526.8,528.5,530.1,531.8,533.5,535.1,536.8,538.5,540.2,541.8,543.5,545.2,546.9,548.5,550.2,551.9,553.6,555.2,556.9,558.6,560.3,561.9,563.6,565.3,566.9,568.6,570.3,572,573.6,575.3,
577,578.7,580.3,582,583.7,585.4,587,588.7,590.4,592.1,593.7,595.4,597.1,598.7,600.4,602.1,603.8,605.4,607.1,608.8,610.5,612.1,613.8,615.5,617.2,618.8,620.5,622.2,623.8,625.5,627.2,628.9,630.5,632.2,633.9,635.6,637.2,638.9,640.6,642.3,643.9,645.6,647.3,649,650.6,652.3,654,655.6,657.3,659,660.7,662.3,664,665.7,667.4,669,670.7,672.4,674.1,675.7,677.4,679.1,680.8,682.4,684.1,685.8,687.4,689.1,690.8,692.5,694.1,695.8,697.5,699.2,700.8,702.5,704.2,705.9,707.5,709.2,710.9,712.6,714.2,715.9,717.6,719.2,720.9,722.6,724.3,725.9,727.6,729.3,731,732.6,734.3,736,737.7,
739.3,741,742.7,744.4,746,747.7,749.4,751,752.7,754.4,756.1,757.7,759.4,761.1,762.8,764.4,766.1,767.8,769.5,771.1,772.8,774.5,776.2,777.8,779.5,781.2,782.8,784.5,786.2,787.9,789.5,791.2,792.9,794.6,
796.2,797.9,799.6,801.3,802.9,804.6,806.3,807.9,809.6,811.3,813,814.6,816.3,818,819.7,821.3,823,824.7,826.4,828,829.7,831.4,833.1,834.7,836.4,838.1,839.7,841.4,843.1,844.8,846.4,848.1,849.8,851.5,853.1,854.8,856.5,858.2,859.8,861.5,863.2,864.9,866.5,868.2,869.9,871.5,873.2,874.9,876.6,878.2,879.9,881.6,883.3,884.9,886.6,888.3,890,891.6,893.3,895,896.7,898.3,
900]

# Gain values taken straight from HDR
gain5ms = [0.000000,0.000000,0.000000,0.000000,0.000000,0.006329,0.005422,0.005014,0.005375,0.005376,0.005383,0.005425,0.005480,0.005526,0.005557,0.005558,0.005557,0.005533,0.005505,0.005448,0.005381,0.005319,0.005282,0.005254,0.005249,0.005250,0.005274,0.005275,0.005285,0.005286,0.005282,0.005270,0.005260,0.005245,0.005220,0.005194,0.005182,0.005175,0.005181,0.005191,0.005217,0.005247,0.005259,0.005284,0.005284,0.005271,0.005263,0.005244,0.005236,0.005240,0.005251,0.005268,0.005292,0.005321,0.005385,0.005443,0.005500,0.005543,0.005569,0.005578,0.005564,0.005552,0.005537,0.004404,0.004402,0.004413,0.004419,0.004432,0.004466,0.004517,0.004570,0.004638,0.004724,0.004813,0.004916,0.005048,0.005230,0.005477,0.005794,0.006173,0.006530,0.006853,0.007159,0.007610,0.007219,0.006930,0.006684,0.006422,0.006145,0.005764,0.005346,0.004934,0.004624,0.004448,0.004301,0.004214,0.004138,0.004090,0.004059,0.004032,0.004024,0.004021,0.004034,0.004060,0.004078,0.004110,0.004148,0.004183,0.004207,0.004243,0.004267,0.004284,0.004296,0.004291,0.004280,0.004270,0.004250,0.004226,0.004207,0.004186,0.004168,0.004159,0.004156,0.004153,0.004167,0.004192,0.004235,0.004288,0.004341,0.004396,0.004458,0.004501,0.004540,0.004563,0.004569,0.004560,0.004541,0.004523,0.004488,0.004456,0.004418,0.004379,0.004349,0.004325,0.004323,0.004325,0.004338,0.004362,0.004387,0.004433,0.004477,0.004535,0.004580,0.004638,0.004687,0.004733,0.004775,0.004814,0.004842,0.004874,0.004891,0.004912,0.004916,0.004912,0.004920,0.004915,0.004916,0.004913,0.004909,0.004903,0.004908,0.004916,0.004932,0.004961,0.004995,0.005046,0.005085,0.005147,0.005211,0.005298,0.005374,0.005465,0.005568,0.005672,0.005749,0.005829,0.005900,0.005961,0.005994,0.006023,0.006029,0.004325,0.004349,0.004362,0.004370,0.004388,0.004400,0.004413,0.004433,0.004462,0.004496,0.004533,0.004587,0.004652,0.004719,0.004803,0.004882,0.004968,0.005073,0.005148,0.005230,0.005328,0.005422,0.005504,0.005578,0.005640,0.005721,0.005819,0.005867,0.005929,0.005995,0.006070,0.006109,0.006152,0.006212,0.006280,0.006311,0.006345,0.006381,0.006458,0.006509,0.006547,0.006587,0.006638,0.006717,0.006789,0.006834,0.006870,0.006971,0.007097]
gain10ms = [0.000000,0.000000,0.000000,0.000000,0.000000,0.003165,0.002711,0.002507,0.002687,0.002688,0.002691,0.002713,0.002740,0.002763,0.002778,0.002779,0.002778,0.002766,0.002753,0.002724,0.002690,0.002660,0.002641,0.002627,0.002625,0.002625,0.002637,0.002637,0.002642,0.002643,0.002641,0.002635,0.002630,0.002623,0.002610,0.002597,0.002591,0.002588,0.002591,0.002596,0.002609,0.002623,0.002629,0.002642,0.002642,0.002635,0.002631,0.002622,0.002618,0.002620,0.002626,0.002634,0.002646,0.002661,0.002692,0.002722,0.002750,0.002772,0.002784,0.002789,0.002782,0.002776,0.002768,0.002202,0.002201,0.002207,0.002209,0.002216,0.002233,0.002258,0.002285,0.002319,0.002362,0.002407,0.002458,0.002524,0.002615,0.002738,0.002897,0.003086,0.003265,0.003427,0.003580,0.003805,0.003610,0.003465,0.003342,0.003211,0.003073,0.002882,0.002673,0.002467,0.002312,0.002224,0.002151,0.002107,0.002069,0.002045,0.002030,0.002016,0.002012,0.002011,0.002017,0.002030,0.002039,0.002055,0.002074,0.002092,0.002103,0.002122,0.002134,0.002142,0.002148,0.002145,0.002140,0.002135,0.002125,0.002113,0.002104,0.002093,0.002084,0.002080,0.002078,0.002076,0.002084,0.002096,0.002118,0.002144,0.002170,0.002198,0.002229,0.002250,0.002270,0.002281,0.002285,0.002280,0.002271,0.002261,0.002244,0.002228,0.002209,0.002190,0.002175,0.002162,0.002161,0.002163,0.002169,0.002181,0.002193,0.002216,0.002239,0.002268,0.002290,0.002319,0.002344,0.002367,0.002387,0.002407,0.002421,0.002437,0.002446,0.002456,0.002458,0.002456,0.002460,0.002458,0.002458,0.002456,0.002454,0.002452,0.002454,0.002458,0.002466,0.002481,0.002497,0.002523,0.002543,0.002573,0.002606,0.002649,0.002687,0.002733,0.002784,0.002836,0.002875,0.002915,0.002950,0.002980,0.002997,0.003012,0.003015,0.002162,0.002175,0.002181,0.002185,0.002194,0.002200,0.002207,0.002217,0.002231,0.002248,0.002266,0.002294,0.002326,0.002360,0.002401,0.002441,0.002484,0.002537,0.002574,0.002615,0.002664,0.002711,0.002752,0.002789,0.002820,0.002860,0.002909,0.002933,0.002965,0.002997,0.003035,0.003055,0.003076,0.003106,0.003140,0.003156,0.003172,0.003191,0.003229,0.003255,0.003274,0.003293,0.003319,0.003358,0.003394,0.003417,0.003435,0.003485,0.003549]
gain15ms = [0.000000,0.000000,0.000000,0.000000,0.000000,0.002110,0.001807,0.001671,0.001792,0.001792,0.001794,0.001808,0.001827,0.001842,0.001852,0.001853,0.001852,0.001844,0.001835,0.001816,0.001794,0.001773,0.001761,0.001751,0.001750,0.001750,0.001758,0.001758,0.001762,0.001762,0.001761,0.001757,0.001753,0.001748,0.001740,0.001731,0.001727,0.001725,0.001727,0.001730,0.001739,0.001749,0.001753,0.001761,0.001761,0.001757,0.001754,0.001748,0.001745,0.001747,0.001750,0.001756,0.001764,0.001774,0.001795,0.001814,0.001833,0.001848,0.001856,0.001859,0.001855,0.001851,0.001846,0.001468,0.001467,0.001471,0.001473,0.001477,0.001489,0.001506,0.001523,0.001546,0.001575,0.001604,0.001639,0.001683,0.001743,0.001826,0.001931,0.002058,0.002177,0.002284,0.002386,0.002537,0.002406,0.002310,0.002228,0.002141,0.002048,0.001921,0.001782,0.001645,0.001541,0.001483,0.001434,0.001405,0.001379,0.001363,0.001353,0.001344,0.001341,0.001340,0.001345,0.001353,0.001360,0.001370,0.001383,0.001394,0.001402,0.001414,0.001422,0.001428,0.001432,0.001430,0.001427,0.001423,0.001417,0.001409,0.001402,0.001396,0.001389,0.001386,0.001385,0.001384,0.001389,0.001397,0.001412,0.001429,0.001447,0.001465,0.001486,0.001500,0.001513,0.001521,0.001523,0.001520,0.001514,0.001508,0.001496,0.001485,0.001473,0.001460,0.001450,0.001442,0.001441,0.001442,0.001446,0.001454,0.001462,0.001478,0.001492,0.001512,0.001527,0.001546,0.001562,0.001578,0.001592,0.001605,0.001614,0.001625,0.001630,0.001637,0.001639,0.001637,0.001640,0.001638,0.001639,0.001638,0.001636,0.001634,0.001636,0.001639,0.001644,0.001654,0.001665,0.001682,0.001695,0.001716,0.001737,0.001766,0.001791,0.001822,0.001856,0.001891,0.001916,0.001943,0.001967,0.001987,0.001998,0.002008,0.002010,0.001442,0.001450,0.001454,0.001457,0.001463,0.001467,0.001471,0.001478,0.001487,0.001499,0.001511,0.001529,0.001551,0.001573,0.001601,0.001627,0.001656,0.001691,0.001716,0.001743,0.001776,0.001807,0.001835,0.001859,0.001880,0.001907,0.001940,0.001956,0.001976,0.001998,0.002023,0.002036,0.002051,0.002071,0.002093,0.002104,0.002115,0.002127,0.002153,0.002170,0.002182,0.002196,0.002213,0.002239,0.002263,0.002278,0.002290,0.002324,0.002366]
gain25ms = [0.000000,0.000000,0.000000,0.000000,0.000000,0.001266,0.001084,0.001003,0.001075,0.001075,0.001077,0.001085,0.001096,0.001105,0.001111,0.001112,0.001111,0.001107,0.001101,0.001090,0.001076,0.001064,0.001056,0.001051,0.001050,0.001050,0.001055,0.001055,0.001057,0.001057,0.001056,0.001054,0.001052,0.001049,0.001044,0.001039,0.001036,0.001035,0.001036,0.001038,0.001043,0.001049,0.001052,0.001057,0.001057,0.001054,0.001053,0.001049,0.001047,0.001048,0.001050,0.001054,0.001058,0.001064,0.001077,0.001089,0.001100,0.001109,0.001114,0.001116,0.001113,0.001110,0.001107,0.000881,0.000880,0.000883,0.000884,0.000886,0.000893,0.000903,0.000914,0.000928,0.000945,0.000963,0.000983,0.001010,0.001046,0.001095,0.001159,0.001235,0.001306,0.001371,0.001432,0.001522,0.001444,0.001386,0.001337,0.001284,0.001229,0.001153,0.001069,0.000987,0.000925,0.000890,0.000860,0.000843,0.000828,0.000818,0.000812,0.000806,0.000805,0.000804,0.000807,0.000812,0.000816,0.000822,0.000830,0.000837,0.000841,0.000849,0.000853,0.000857,0.000859,0.000858,0.000856,0.000854,0.000850,0.000845,0.000841,0.000837,0.000834,0.000832,0.000831,0.000831,0.000833,0.000838,0.000847,0.000858,0.000868,0.000879,0.000892,0.000900,0.000908,0.000913,0.000914,0.000912,0.000908,0.000905,0.000898,0.000891,0.000884,0.000876,0.000870,0.000865,0.000865,0.000865,0.000868,0.000872,0.000877,0.000887,0.000895,0.000907,0.000916,0.000928,0.000937,0.000947,0.000955,0.000963,0.000968,0.000975,0.000978,0.000982,0.000983,0.000982,0.000984,0.000983,0.000983,0.000983,0.000982,0.000981,0.000982,0.000983,0.000986,0.000992,0.000999,0.001009,0.001017,0.001029,0.001042,0.001060,0.001075,0.001093,0.001114,0.001134,0.001150,0.001166,0.001180,0.001192,0.001199,0.001205,0.001206,0.000865,0.000870,0.000872,0.000874,0.000878,0.000880,0.000883,0.000887,0.000892,0.000899,0.000907,0.000917,0.000930,0.000944,0.000961,0.000976,0.000994,0.001015,0.001030,0.001046,0.001066,0.001084,0.001101,0.001116,0.001128,0.001144,0.001164,0.001173,0.001186,0.001199,0.001214,0.001222,0.001230,0.001242,0.001256,0.001262,0.001269,0.001276,0.001292,0.001302,0.001309,0.001317,0.001328,0.001343,0.001358,0.001367,0.001374,0.001394,0.001419]


In [ ]:
############################################################################################################
# This cell:
# Takes a root directory of calibration standard images (cropped) and computes a radiance value avg sheet.
# IMPORTANT:
# On execution make sure to change: Output_file_name AND exposure AND gainarray

# Left To Do:
## IOP1: REDO
## IOP2: 
## IOP3: 
## IOP4: 
## IOP5: 
## IOP6: 

## Needs to be completed:
## IOP1: 25ms
## IOP2: 5ms, 10ms, 15ms, 25ms
## IOP3: 5ms, 10ms, 15ms, 25ms
## IOP4: 5ms, 10ms, 15ms, 25ms
## IOP5: 5ms, 10ms, 15ms, 25ms
## IOP6: 5ms, 10ms, 15ms, 25ms


###################################################################
# Change these variables                                          #
###################################################################
exposure = '_25ms'                                                 #
path_RootFolder =  "Z:\\Data\\St_Supery\\IOP1_Sep2020\\Tripod"       #
Output_File_Name = f"IOP1_Cropped_STD_RadianceAverage_{exposure}" #
###################################################################
## Finding all the HDRs and BINs
# list of all the folders found:
list_stdFolders = [join(path_RootFolder,x) for x in listdir(path_RootFolder)]
# print(f"collected folders: {list_stdFolders}")

# The name of the folders to use as row index:
list_std_names = [x for x in listdir(path_RootFolder)]


table_radianceRows = [] 
list_STD_NamesUsed = []
for idx_folder, stdFolder in enumerate(list_stdFolders):
    #
    ## Load the data of an image:
    #
    list_radianceAvg = []
    path_bin, path_hdr = Extract_Files_By_Exposure(stdFolder,exposure)
    
    if isfile(path_hdr) and isfile(path_bin):
        # Load the image, track which file loaded correctly, track its 2d area
        try:
            print(f'Loading: {idx_folder,list_std_names[idx_folder]}')
            envi_hsi = envi.open(path_hdr,image=path_bin)
            hcube = np.array(envi_hsi.load())
            totalPixels = len(hcube) * len(hcube[0])
            list_STD_NamesUsed.append(list_std_names[idx_folder])

        # Choose the gain values for calculation
            if exposure == '_5ms':
                gains = gain5ms
            elif exposure == '_10ms':
                gains = gain10ms
            elif exposure == '_15ms':
                gains = gain15ms
            else:
                gains = gain25ms
        except:
            print(f"skipping error on hypercube loading , {idx_folder,stdFolder}\\n")
    else:
        # Errors could be: network speed  
        print(f"skipping: {idx_folder,stdFolder} , potentially \\n no image exists for this exposure")
        continue
    #
    ## Calculate row values 
    #
    for band in range(len(wv)):
        try:
            appliedGain = np.multiply(hcube[:,:,band] , gains[band]) 
            sumRadiance = np.sum( appliedGain ) # sum of all pixel-gain values
            list_radianceAvg.append(  sumRadiance / totalPixels )
        except: 
            # Error logs:
            # hypercube missing several bands, append NaN to these columns
            print(f'appending NaN')
            list_radianceAvg.append(np.nan)

    # Add the row to the table
    table_radianceRows.append(list_radianceAvg)
#
## Generate output csv
#
df_outputsheet = pd.DataFrame(data=table_radianceRows,columns=wv)
df_outputsheet.insert(0,'STD_ID',list_STD_NamesUsed)
df_outputsheet.to_csv(f'Tripod_{Output_File_Name}.csv',index = False)  
print("complete")
winsound.PlaySound('sound.wav', winsound.SND_FILENAME)

#### This next cell is identical to the one above, but automated over all exposures of a folder. Due to networking speeds, i could not test it thoroughly

In [ ]:
# testing a loop on exposures:
############################################################################################################
# This cell:
# Takes an nested-iteration approach to the cell above. Due to networking speeds, could not properly test this.
# It will take simply a root directory of calibration standard images (cropped) and computes a radiance value avg sheet.
# IMPORTANT:
# On execution make sure to change: path to the parent/root directory


###################################################################
# Change these variables                                          #
###################################################################                                                #
path_RootFolder =  "Z:\\Data\\St_Supery\\IOP6_October2021\\Tripod"    #
###################################################################
list_exposures = ['_5ms','_10ms','_15ms','_25ms'] 
for exposure in list_exposures:
    if "IOP1" in path_RootFolder:
        tripodName = "IOP1"
    elif "IOP2" in path_RootFolder:
        tripodName = "IOP2"
    elif "IOP3" in path_RootFolder:
        tripodName = "IOP3"
    elif "IOP4" in path_RootFolder:
        tripodName = "IOP4"
    else : 
        tripodName = "Path_Error_"



    Output_File_Name = f"{tripodName}_Cropped_STD_RadianceAverage_{exposure}" #

## Finding all the HDRs and BINs
# list of all the folders found:
    list_stdFolders = [join(path_RootFolder,x) for x in listdir(path_RootFolder) if x[-3:] == 'STD']
# The name of the folders to use as row index:
    list_std_names = [x for x in listdir(path_RootFolder) if x[-3:] == 'STD']


    table_radianceRows = [] 
    list_STD_NamesUsed = []
    for idx_folder, stdFolder in enumerate(list_stdFolders):
    #
    ## Load the data of an image:
    #
        list_radianceAvg = []
        path_bin, path_hdr = Extract_Files_By_Exposure(stdFolder,exposure)
    
        if isfile(path_hdr) and isfile(path_bin):
            # Load the image, track which file loaded correctly, track its 2d area
            try:
                print(f'Loading: {idx_folder,list_std_names[idx_folder]}')
                envi_hsi = envi.open(path_hdr,image=path_bin)
                hcube = np.array(envi_hsi.load())
                totalPixels = len(hcube) * len(hcube[0])
                list_STD_NamesUsed.append(list_std_names[idx_folder])

        # Choose the gain values for calculation
                if exposure == '_5ms':
                    gains = gain5ms
                elif exposure == '_10ms':
                    gains = gain10ms
                elif exposure == '_15ms':
                    gains = gain15ms
                else:
                    gains = gain25ms
            except:
                print(f"skipping error on hypercube loading , {idx_folder,stdFolder}\\n")                
        else:
        # Errors could be: network speed  
            print(f"skipping: {idx_folder,stdFolder} , potentially \\n no image exists for this exposure")
            continue
    #
    ## Calculate row values 
    #
        for band in range(len(wv)):
            try:
                appliedGain = np.multiply(hcube[:,:,band] , gains[band]) 
                sumRadiance = np.sum( appliedGain ) # sum of all pixel-gain values
                list_radianceAvg.append(  sumRadiance / totalPixels )
            except: 
            # Error logs:
            # hypercube missing several bands
                print(f'appending NaN')
                list_radianceAvg.append(np.nan)

    # Add the row to the table
        table_radianceRows.append(list_radianceAvg)
#
## Generate output csv
#
    df_outputsheet = pd.DataFrame(data=table_radianceRows,columns=wv)
    df_outputsheet.insert(0,'STD_ID',list_STD_NamesUsed)
    df_outputsheet.to_csv(f'Tripod_{Output_File_Name}.csv',index = False)  
    print("complete")
    winsound.PlaySound('sound.wav', winsound.SND_FILENAME)

In [ ]:
#
## Trouble shooting printouts for the cell above
#

# print(hcube.shape)
# print(f"Amount of pixels {totalPixels}")
# print(f"bands: {len(wv)}")
# print(f"Rows: {len(hcube)} , Columns: {len(hcube[0])}")
# print(f"Length of rad_avg: {len(list_radianceAvg)}")
# print(list_radianceAvg[-1:]) # last in the list
# print(f"current size of output table: {len(table_radianceRows)}")
# print(table_radianceRows)

#### This cell grabs the radiance meter values

In [23]:
###################################################################################################################################################################################################
## Overview:
# Executing this cell will produce a spreadsheet of all radiance averages provided by a radiance meter for a given path to a hyperspectral image collection (Parent Directory).
# First, it loops through all calibration standards.
# Then it log the values stored from mounted radiance meter: solar irradiance
###################################################################################################################################################################################################
# ASSUMES: subdirectory \RadianceSheets exists
# 
# The wavelength values consistent among all exposure lengths
wv = [500,501.7,503.3,505,506.7,508.4,510,511.7,513.4,515.1,516.7,518.4,520.1,521.8,523.4,525.1,526.8,528.5,530.1,531.8,533.5,535.1,536.8,538.5,540.2,541.8,543.5,545.2,546.9,548.5,550.2,551.9,553.6,555.2,556.9,558.6,560.3,561.9,563.6,565.3,566.9,568.6,570.3,572,573.6,575.3,
577,578.7,580.3,582,583.7,585.4,587,588.7,590.4,592.1,593.7,595.4,597.1,598.7,600.4,602.1,603.8,605.4,607.1,608.8,610.5,612.1,613.8,615.5,617.2,618.8,620.5,622.2,623.8,625.5,627.2,628.9,630.5,632.2,633.9,635.6,637.2,638.9,640.6,642.3,643.9,645.6,647.3,649,650.6,652.3,654,655.6,657.3,659,660.7,662.3,664,665.7,667.4,669,670.7,672.4,674.1,675.7,677.4,679.1,680.8,682.4,684.1,685.8,687.4,689.1,690.8,692.5,694.1,695.8,697.5,699.2,700.8,702.5,704.2,705.9,707.5,709.2,710.9,712.6,714.2,715.9,717.6,719.2,720.9,722.6,724.3,725.9,727.6,729.3,731,732.6,734.3,736,737.7,
739.3,741,742.7,744.4,746,747.7,749.4,751,752.7,754.4,756.1,757.7,759.4,761.1,762.8,764.4,766.1,767.8,769.5,771.1,772.8,774.5,776.2,777.8,779.5,781.2,782.8,784.5,786.2,787.9,789.5,791.2,792.9,794.6,
796.2,797.9,799.6,801.3,802.9,804.6,806.3,807.9,809.6,811.3,813,814.6,816.3,818,819.7,821.3,823,824.7,826.4,828,829.7,831.4,833.1,834.7,836.4,838.1,839.7,841.4,843.1,844.8,846.4,848.1,849.8,851.5,853.1,854.8,856.5,858.2,859.8,861.5,863.2,864.9,866.5,868.2,869.9,871.5,873.2,874.9,876.6,878.2,879.9,881.6,883.3,884.9,886.6,888.3,890,891.6,893.3,895,896.7,898.3,
900]

# FOR REFERENCE: These are the absolute paths from my computer:

# IOP1 # "V:\\Data\\St_Supery\\IOP1_Sep2020\\Tripod"
# IOP2 # "V:\\Data\\St_Supery\\IOP2_Oct2020\\Tripod"
# IOP3 # "V:\\Data\\St_Supery\\IOP3_July2021\\Tripod" 
# IOP4 # "V:\\Data\\St_Supery\\IOP4_August2021\\Tripod"
# IOP5 # "V:\\Data\\St_Supery\\‭IOP5_Sep2021\\Tripod"
# IOP6 # "V:\\Data\\St_Supery\\IOP6_October2021\\tripod"
# The absolute path to the parent directory of all the hyperspectral images

###################################################################################################################################################################################################
## Change these variables:
# This should be the parent directory to the Tripod image dataset
rawHDRPath = "V:\\Data\\St_Supery\\IOP4_August2021\\Tripod"
# What you choose to name the output spreadsheet
outputName = "IOP4_Tripod_10ms"
###################################################################################################################################################################################################


imageFolders = [join(rawHDRPath, folder) for folder in listdir(rawHDRPath) if folder[-3:] == 'STD' ] 

if "_5ms" in outputName:
    subPath = "canopy_lb_2020_5ms\\canopy_lb_2020_5ms_000000\\canopy_lb_2020_5ms_000000.hdr"
elif "_10ms" in outputName:
    subPath = "canopy_lb_2020_10ms\\canopy_lb_2020_10ms_000000\\canopy_lb_2020_10ms_000000.hdr"
elif "_15ms" in outputName:
    subPath = "canopy_lb_2020_15ms\\canopy_lb_2020_15ms_000000\\canopy_lb_2020_15ms_000000.hdr"
elif "_25ms" in outputName:
    subPath = "canopy_lb_2020_25ms\canopy_lb_2020_25ms_000000\\canopy_lb_2020_25ms_000000.hdr"
else:
    print("wtf")



solarIrradiance = []
imName = []
# ATTN : if your local directory requires more/less characters the names within columns may be cut off short/long
if 'IOP4_August2021' in rawHDRPath:
    nameIndex = 41 
else:
    nameIndex = 39

# Load the image data, collect the values in field : solar irradiance, log it in a spreadsheet
for i in range(len(imageFolders)):

    # print(f'Iteration: {i} filename: {join(imageFolders[i],subPath)}')
    try:
        hdrFile = join(imageFolders[i],subPath)
        if isfile(hdrFile):
            radiance = get_solar_irradiance_from_hdr_path(hdrFile)
            solarIrradiance.append(radiance)
            imName.append(imageFolders[i][nameIndex:]) # IOP4 requires 41, 39 all others
        else:
            # default values incase none could be extracted from hdr
            radiance = np.zeros(240)
            solarIrradiance.append(radiance)
            imName.append(imageFolders[i][nameIndex:])
    except:
        # Catches errors for networking, corrupted data file
        print(f"error on {imageFolders[i]}")
        radiance = np.zeros(240)
        solarIrradiance.append(radiance)
        imName.append(imageFolders[i][nameIndex:])
        pass
df_meterRadiance = pd.DataFrame(data=solarIrradiance, columns=wv)
df_meterRadiance.insert(0,'img_id',imName)
df_meterRadiance.head()
df_meterRadiance.to_csv(f'{outputName}_Tripod_RadianceMeter.csv',index = False)


In [ ]:
# Debug printouts:
# print(f"length of imagefolders : {len(imageFolders)}")
# print(f"Length of collected irradiance: {len(solarIrradiance)}")
# work = get_solar_irradiance_from_hdr_path(join(imageFolders[0],subPath))
